# CNN  - This is a Convolutional Neural Network example using Keras

    Copyright (C) 2020 Adrian Bevan, and 2023, 2024 Abbey Waldron
    Queen Mary University of London

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.
    
----------------------

CNN example using a Keras model. This is a simple example of a 2-conv layer with max pooling CNN example that has fully connected (Dense) layers following prior to the output prediction. This example is used for processing the MNIST (or CFAR10) data using the Adam optimiser. 

For more information about the Adam optimiser please see the  paper by Kingma and Ba, [arXiv:1412.6980](https://arxiv.org/abs/1412.6980).

## Useful background on MNIST and data wrangling

Set `UseMNIST = True` to train on the MNIST data.

The MNIST data is a set of 60,000 training and 10,000 test examples.  Each example is a hand written integer between 0 and 9, represented by a greyscale image with 28x28 pixels. This constitutes a 784 dimensional input feature space, where each image is a number between 0 and 255 (8 bit greyscale).  
 
For more information about the MNIST data set please see the website http://yann.lecun.com/exdb/mnist/.

## Useful background on CFAR10

Set `UseMNIST = False` to train on the CFAR10 data.

The CFAR10 data is a set of 60000 training and 50000 test examples. Each example is one of 10 categories, and is represented by colour image with 32x32 pixels.  This constitutes a 3072 dimensional input feature space (32x32x3), where each feature is an 8bit colour representation, i.e. a number between 0 and 255.  The 10 categories are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck.

For more information about the CFAR10 (and CFAR100) data set please see the website https://www.cs.toronto.edu/~kriz/cifar.html.


## Data Wrangling

In order for an image to be processed efficiently by a neural network the pixel colour in the range $[0, 255]$ is mapped into the domain $[0, 1]$, following the usual [Efficient Backpropagation](http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf) data wrangling guidelines. For an MLP the 28x28 pixel image features (i.e. the pixels) are considered a linear set of 784 features that are input to the network.


----------------------
## Load and pre-process the data

The MNIST data are directly accessible via keras as a dataset. So we first load the data. As noted above (and as you will have explored in the linear regression example), it is important to closely match the weights used in a network with the feature space domain that is being studied, so that the optimiser has less work to do in order to converge to the optimal solution.  In this case we achieve that by mapping the 8-bit greyscale color value $[0, 255]$ on the domain $[0, 1]$.

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras

def getCFAR10():
    print("\033[92mLoading the CFAR10 data from keras\033[0m")
    # Load the CFAR10 data via the tensorflow keras dataset:
    cifar10 = keras.datasets.cifar10
    ... = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    return x_train, y_train, x_test, y_test

def getMNIST():
    print("\033[92mLoading the MNIST data from keras\033[0m")
    # Load the MNIST data via the tensorflow keras dataset:
    mnist = keras.datasets.mnist
    ... = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    return x_train, y_train, x_test, y_test

#UseMNIST = False 
UseMNIST = True 

x_train = None
y_train = None
x_test  = None
y_test  = None
if UseMNIST:
    ... = getMNIST()
    x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
    x_test  = x_test.reshape((x_test.shape[0], 28, 28, 1))

else:
    ... = getCFAR10()

print("\tN(train)             = ", len(x_train))
print("\tN(test)              = ", len(x_test))
print("\tTest data shape      = ", x_train.shape )
print("\tTrain data shape     = ", x_test.shape )


---------------------
## Building the model 

**Training [Epochs, batches and validation data]** The number of training epochs specified is denoted by `Nepochs`.  1 epoch is required to run over all of the training data.  It is possible to run batches or mini-batches of data through the training; each batch requires the optimisation to be performed, and so when specifying the `BatchSize` the training will be performed by running the optimisation $N_{epochs}\times N_{batches}$ times.  In general this leads to faster optimisation of the model than optimising over the full training set each time.

The training data will be split into training and validation samples according to the value of the variable `ValidationSplit`. 

**Dropout:** Coadaptation is the ability for an optimisation algorithm to allow weights to be learned where changes in one node can be compensated by changes in another node that limit the increase in performance.  This issue can be a problem for deep networks in particular where the optimisation process can involve millions of hyperparameters.  A way to combat this issue is to randomly drop-out nodes in the network each iteration of the optimisation.  That way no single paring of nodes can learn to co-adapt to the evolution of hyperparameters through the optimisation.  Thankfully all the user has to do is to set a dropout value via the variable `DropoutValue`. 

**NOTE:** this value is the fraction of nodes dropped from the model.

**Loss:** The cross entropy loss function is used for this optimisation process.  The value of the loss function is converted into an output vector of 1's and 0's to be used for classification.

### Model configuration



In [ ]:
#
# Training configuration
#
ValidationSplit = 0.5
BatchSize       = 10 # be careful with this, if it is too big may crash!
Nepochs         = ...
DropoutValue    = ...

#
# as CFAR10 and MNIST are different shapes, extract the shape parameters to properly
# configure the CNN input data
#
xpix = x_train.shape[1]
ypix = x_train.shape[2]
zpix = x_train.shape[3] # color channels


model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(xpix, ypix, zpix)))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

print("--------------------------------------------------------------------------------------------------------------")
print("\033[92mWill train a convolutional neural network on the MNIST data\033[0m")
print("--------------------------------------------------------------------------------------------------------------\n\n")
print("Input data MNIST")
print("Dropout values       = ", DropoutValue)
print("Leaky relu parameter =  0.1")
print("ValidationSplit      = ", ValidationSplit)
print("BatchSize            = ", BatchSize)
print("Nepochs              = ", Nepochs, "\n")
print("N(train)             = ", len(x_train))
print("N(test)              = ", len(x_test))
model.summary()

# now specify the loss function - cross entropy
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# now we can train the model to make predictions.
#   Use the ADAM optimiser
#   Specify the metrics to report as accuracy
#   Specify the loss function (see above)
# the fit step specifies the number of training epochs
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
history  = model.fit(..., ..., validation_split=..., batch_size=..., epochs=...)


------------
## Output

When training a model we are interested in studying the accuracy of prediction (in this case how often do we correctly classify an MNIST image according to its true label), and about the evolution of the loss function for both the test and train samples of examples.  If the test and train sample loss functions are signficantly different that points toward a problem... the model could be under or over trained.

This information is stored in the 'history' variable.

In [ ]:
import matplotlib.pyplot as plt

# Print out the history keys expected are:
#    loss        The loss function evaluated at each epoch for the training set
#    acc         The accuracy evaluated at each epoch for the training set
#    val_loss    The loss evaluated at each epoch for the validation set
#    val_acc     The accuracy evaluated at each epoch for the validation set
# The val_* entries exist only if there is a validation_split specified

print("history keys = ", history.history.keys())

print("\n\033[1mDisplay the evolution of the accuracy as a function of the training epoch\033[0m")
print("  N(Epochs)        = ", Nepochs)

print("  accuracy (train) = ", history.history['accuracy'])
print("  accuracy (test)  = ", history.history['val_accuracy'])
plt.plot(...) # 'accuracy'
plt.plot(...) # 'val_accuracy'
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='lower right')
plt.show()
plt.clf()

print("\n\033[1mDisplay the evolution of the loss as a function of the training epoch\033[0m")
print("  N(Epochs)        = ", Nepochs)
print("  loss (train)     = ", history.history['loss'])
print("  loss (test)      = ", history.history['val_loss'])

# summarize history for loss
plt.plot(...) # 'loss'
plt.plot(...) # 'val_loss'
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper right')
plt.show()

# having finished training the model, use this to evaluate the performance on a sample of test data
print("\n\033[1mPerformance summary (on test data):\033[0m")
loss, acc = model.evaluate(...,  ..., verbose=2) # remember to use test set!
print("\tloss = {:5.3f}\n\taccuracy = {:5.3f}".format(loss, acc))


----------------------
## Suggested exercises:

 - Explore the effect of DropOut, ValidationSplit, Nepochs, and BatchSize have on the training (try to find a model where the test and train loss function values are similar.
 - Explore how the CNN affects the training performance e.g. 
   - change the number of convolution filters in each layer.  The current values of these are 32, 64 and 64
   - change the number of nodes in the fully connected (Dense) layer. The current value of nodes in this layer is 64.
 - Explore the effect of adding a second dropout layers into the network after the conv and Dense layers (see the NN.ipynb example for how to implement a dense layer in a model.
 - What is the best performance you can get on each dataset?
 - Plot some of the MNIST and CFAR10 examples that are classified incorrectly (hint: use model.predict and matplotlib's imshow).  Was it easy for you to classify them by eye?